In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Reshape
import numpy as np
import cv2
import os

input_shape = (224, 224, 3)  
num_boxes = 5

model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))

model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

model.summary()

def preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (input_shape[0], input_shape[1]))
    image = image / 255.0  
    return image

def preprocess_labels(label_path):
    with open(label_path, 'r') as f:
        label_str = f.read().strip()

    labels = []
    for label in label_str.split('\n'):
        if label.strip():
            label_data = label.split()
            class_id = int(label_data[0])
            x = float(label_data[1])
            y = float(label_data[2])
            w = float(label_data[3])
            h = float(label_data[4])
            labels.append([class_id, x, y, w, h])

    labels = labels[:num_boxes]  
    labels.extend([[0, 0, 0, 0, 0]] * (num_boxes - len(labels)))  

  
    labels = np.array(labels)

    return labels

def load_data(image_dir, label_dir):
    image_paths = sorted([os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith('.jpg') or f.endswith('.png')])
    label_paths = sorted([os.path.join(label_dir, f) for f in os.listdir(label_dir) if f.endswith('.txt')])

    images = np.array([preprocess_image(path) for path in image_paths])
    labels = np.array([preprocess_labels(path) for path in label_paths])

    return images, labels

train_images_dir = r'C:\Users\pbsns\OneDrive\Documents\cancer\Leaukemia Detection\train\images'
train_labels_dir = r'C:\Users\pbsns\OneDrive\Documents\cancer\Leaukemia Detection\train\labels'
val_images_dir = r'C:\Users\pbsns\OneDrive\Documents\cancer\Leaukemia Detection\valid\images'
val_labels_dir =  r'C:\Users\pbsns\OneDrive\Documents\cancer\Leaukemia Detection\valid\labels'

train_images, train_labels = load_data(train_images_dir, train_labels_dir)
val_images, val_labels = load_data(val_images_dir, val_labels_dir)

max_label_elements = max(len(label) for label in train_labels)

output_size = max_label_elements * num_boxes

model.add(Dense(output_size, activation='linear'))
model.add(Reshape((num_boxes, max_label_elements)))  

model.fit(train_images, train_labels, epochs=10, batch_size=32, validation_data=(val_images, val_labels))


C:\Users\pbsns\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 52, 52, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 26, 26, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 86528)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │      11,075,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 64)                  │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 11,177,216 (42.64 MB)

 Trainable params: 11,177,216 (42.64 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
206/206 ━━━━━━━━━━━━━━━━━━━━ 169s 787ms/step - accuracy: 0.4616 - loss: 1.0674 - val_accuracy: 0.8534 - val_loss: 0.2955
Epoch 2/10
206/206 ━━━━━━━━━━━━━━━━━━━━ 155s 754ms/step - accuracy: 0.8352 - loss: 0.3507 - val_accuracy: 0.8534 - val_loss: 0.2653
Epoch 3/10
206/206 ━━━━━━━━━━━━━━━━━━━━ 149s 722ms/step - accuracy: 0.8444 - loss: 0.2576 - val_accuracy: 0.8534 - val_loss: 0.1729
Epoch 4/10
206/206 ━━━━━━━━━━━━━━━━━━━━ 140s 681ms/step - accuracy: 0.8575 - loss: 0.2097 - val_accuracy: 0.8572 - val_loss: 0.1573
Epoch 5/10
206/206 ━━━━━━━━━━━━━━━━━━━━ 146s 707ms/step - accuracy: 0.8559 - loss: 0.1934 - val_accuracy: 0.8601 - val_loss: 0.1269
Epoch 6/10
206/206 ━━━━━━━━━━━━━━━━━━━━ 148s 716ms/step - accuracy: 0.8511 - loss: 0.1733 - val_accuracy: 0.8588 - val_loss: 0.1455
Epoch 7/10
206/206 ━━━━━━━━━━━━━━━━━━━━ 150s 728ms/step - accuracy: 0.8562 - loss: 0.1702 - val_accuracy: 0.8248 - val_loss: 0.1263
Epoch 8/10
206/206 ━━━━━━━━━━━━━━━━━━━━ 149s 723ms/step - accuracy: 0.8496 -

In [2]:
from tensorflow.keras.models import save_model

save_model(model, 'luk_cnn_model.keras') 

In [3]:
import os
import cv2
import numpy as np


def preprocess_image(image_path, input_shape):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (input_shape[0], input_shape[1]))
    image = image / 255.0 
    return image


def load_data(image_dir, label_dir, input_shape):
    image_paths = sorted([os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith('.jpg') or f.endswith('.png')])
    label_paths = sorted([os.path.join(label_dir, f) for f in os.listdir(label_dir) if f.endswith('.txt')])
    images = np.array([preprocess_image(path, input_shape) for path in image_paths])
    labels = np.array([preprocess_labels(path) for path in label_paths])
    return images, labels


test_images_dir = r'C:\Users\pbsns\OneDrive\Documents\cancer\Leaukemia Detection\test\images'
test_labels_dir = r'C:\Users\pbsns\OneDrive\Documents\cancer\Leaukemia Detection\test\labels'


input_shape = (224, 224, 3)


test_images, test_labels = load_data(test_images_dir, test_labels_dir, input_shape)

loss, accuracy = model.evaluate(test_images, test_labels)

print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')


10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 191ms/step - accuracy: 0.6753 - loss: 0.1214
Test Loss: 0.1202058419585228
Test Accuracy: 0.8416666984558105


In [4]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score

def calculate_iou(box1, box2):
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])

    intersection_area = max(0, x2 - x1 + 1) * max(0, y2 - y1 + 1)

    box1_area = (box1[2] - box1[0] + 1) * (box1[3] - box1[1] + 1)
    box2_area = (box2[2] - box2[0] + 1) * (box2[3] - box2[1] + 1)

    iou = intersection_area / float(box1_area + box2_area - intersection_area)
    return iou

predictions = model.predict(test_images)

predictions_flat = predictions.reshape(-1, num_boxes * max_label_elements)
test_labels_flat = test_labels.reshape(-1, num_boxes * max_label_elements)

threshold = 0.5
binary_predictions = (predictions_flat > threshold).astype(int)
binary_test_labels = (test_labels_flat > threshold).astype(int)

precision = precision_score(binary_test_labels, binary_predictions, average='weighted')
recall = recall_score(binary_test_labels, binary_predictions, average='weighted')
f1 = f1_score(binary_test_labels, binary_predictions, average='weighted')

print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1-score: {f1}')

ious = []
for i in range(len(predictions)):
    for j in range(num_boxes):
        pred_box = predictions[i][j]
        true_box = test_labels[i][j][1:5]  
        iou = calculate_iou(pred_box, true_box)
        ious.append(iou)

average_iou = np.mean(ious)
print(f'Average IoU: {average_iou}')

10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 203ms/step
Precision: 0.6836181493878984
Recall: 0.5536431440045898
F1-score: 0.5395145191845682
Average IoU: 0.09006635726964037


C:\Users\pbsns\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [9]:
import cv2
import numpy as np

def preprocess_image(image_path, input_shape):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (input_shape[0], input_shape[1]))
    image = image / 255.0 
    return image

input_shape = (224, 224, 3)

test_image_path = 'C:\\Users\\pbsns\\OneDrive\\Documents\\cancer\\Leaukemia Detection\\test\\images\\WBC-Benign-079_jpg.rf.fefc85c808ca8eb74393a40b0c8b64ea.jpg'

test_image = preprocess_image(test_image_path, input_shape)

test_image = np.expand_dims(test_image, axis=0)

predictions = model.predict(test_image)

print(predictions)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
[[[0.4777201  0.53609794 0.5256649  0.13257144 0.15088268]
  [0.45297757 0.5103664  0.51154834 0.13545382 0.15427749]
  [0.40333164 0.42813635 0.4646268  0.13737525 0.15678643]
  [0.38811383 0.42385885 0.4259312  0.12241807 0.1444227 ]
  [0.3435279  0.36051273 0.36574686 0.107972   0.13641159]]]
